# Notebook 02 — Build Dataset (Reproducible)

This notebook produces processed datasets and portfolio-ready visuals from raw Eurostat `nrg_bal_peh` exports.


In [ ]:
import pandas as pd
import numpy as np
import os

from google.colab import files
uploaded = files.upload()

RAW_FILE = list(uploaded.keys())[0]
df = pd.read_csv(RAW_FILE)

#  Keep only columns we need (defensive)
keep_cols = ["geo", "TIME_PERIOD", "siec", "OBS_VALUE"]
df = df[keep_cols].copy()

#  Numeric + missing -> 0
df["OBS_VALUE"] = pd.to_numeric(df["OBS_VALUE"], errors="coerce").fillna(0)
df["TIME_PERIOD"] = df["TIME_PERIOD"].astype(int)

#  Filter to electricity-relevant fuel types
valid_sources = [
    "Solid fossil fuels",
    "Natural gas",
    "Nuclear heat",
    "Primary solid biofuels",
    "Biogases",
    "Hydro",
    "Geothermal",
    "Wind",
    "Non-renewable waste",
    "Renewable municipal waste"
]
df_power = df[df["siec"].isin(valid_sources)].copy()

#  Map Eurostat fuels -> human energy groups
energy_map = {
    "Solid fossil fuels": "Coal",
    "Natural gas": "Natural Gas",
    "Nuclear heat": "Nuclear",
    "Hydro": "Hydro",
    "Wind": "Wind",
    "Geothermal": "Geothermal",
    "Primary solid biofuels": "Bioenergy",
    "Biogases": "Bioenergy",
    "Renewable municipal waste": "Bioenergy",
    "Non-renewable waste": "Bioenergy"
}
df_power["energy_group"] = df_power["siec"].map(energy_map)

#  Aggregate generation (GWh) by country-year-energy_group
df_grouped = (
    df_power
    .groupby(["geo", "TIME_PERIOD", "energy_group"], as_index=False)["OBS_VALUE"]
    .sum()
    .rename(columns={"OBS_VALUE": "generation_gwh"})
)

#  Totals + shares
totals = (
    df_grouped
    .groupby(["geo", "TIME_PERIOD"], as_index=False)["generation_gwh"]
    .sum()
    .rename(columns={"generation_gwh": "total_gwh"})
)

df_grouped = df_grouped.merge(totals, on=["geo", "TIME_PERIOD"], how="left")
df_grouped["share"] = np.where(df_grouped["total_gwh"] > 0,
                               df_grouped["generation_gwh"] / df_grouped["total_gwh"],
                               0)

#  Dominant source per country-year
dominant_source = (
    df_grouped
    .sort_values(["geo", "TIME_PERIOD", "share"], ascending=[True, True, False])
    .groupby(["geo", "TIME_PERIOD"], as_index=False)
    .first()
)

df_grouped.head(), dominant_source.head()


Saving eurostat_generation.csv to eurostat_generation (1).csv


/tmp/ipython-input-3093663859.py:10: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(RAW_FILE)


(       geo  TIME_PERIOD energy_group  generation_gwh  total_gwh  share
 0  Albania         2000    Bioenergy             0.0     4594.0    0.0
 1  Albania         2000         Coal             0.0     4594.0    0.0
 2  Albania         2000   Geothermal             0.0     4594.0    0.0
 3  Albania         2000        Hydro          4594.0     4594.0    1.0
 4  Albania         2000  Natural Gas             0.0     4594.0    0.0,
        geo  TIME_PERIOD energy_group  generation_gwh  total_gwh  share
 0  Albania         2000        Hydro          4594.0     4594.0    1.0
 1  Albania         2001        Hydro          3555.0     3555.0    1.0
 2  Albania         2002        Hydro          3512.0     3512.0    1.0
 3  Albania         2003        Hydro          4885.0     4885.0    1.0
 4  Albania         2004        Hydro          5466.0     5466.0    1.0)

In [ ]:
os.makedirs("data_processed", exist_ok=True)

df_grouped.to_csv("data_processed/generation_by_source_country_year.csv", index=False)
dominant_source.to_csv("data_processed/dominant_source_country_year.csv", index=False)

from google.colab import files
files.download("data_processed/generation_by_source_country_year.csv")
files.download("data_processed/dominant_source_country_year.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>